## validate wrf LLJ detection with lidar LLJ detection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
# read in 0.5 m/s data
s = pd.read_csv('summary1.csv').drop(columns=['Unnamed: 0'])
# s = pd.read_csv('summary1.csv').drop(columns=['Unnamed: 0'])

### start with assessment of hits/misses

In [3]:
tn, fp, fn, tp = confusion_matrix(s.lidar_LLJ, s.wrf_LLJ).ravel()
tn = 8516-len(s) # no true negatives in dataset
print('True negative:', tn, '(WRF predicts no LLJ, there is no LLJ)')
print('True Positive:', tp, '(WRF predicts an LLJ, there is an LLJ)')
print('False positive:', fp, '(WRF predicts an LLJ, there is no LLJ)')
print('False negative:', fn, '(WRF predicts no LLJ, there is an LLJ)')
print('')
print('observed LLJs:', tp+fn, '; forecasted LLJs:', tp+fp)
print('observed no:', tn+fp, '; forecasted no:', tn+fn)

True negative: 8305 (WRF predicts no LLJ, there is no LLJ)
True Positive: 43 (WRF predicts an LLJ, there is an LLJ)
False positive: 66 (WRF predicts an LLJ, there is no LLJ)
False negative: 102 (WRF predicts no LLJ, there is an LLJ)

observed LLJs: 145 ; forecasted LLJs: 109
observed no: 8371 ; forecasted no: 8407


In [4]:
ttl = tn+tp+fn+fp

### Accuracy: what fraction of the forecasts were correct?
- can be misleading because heavily influenced by most common category (no LLJ)

In [5]:
print(f'{((tp+tn)/ttl)*100:.2f}% of all forecasts were correct')

98.03% of all forecasts were correct


### Bias: How did the forecast frequency of "yes" events compare to the observed frequency of "yes" events?

In [6]:
print(f'bias = {((tp+fp)/(tp+fn))}')

bias = 0.7517241379310344


WRF tends to underforecast LLJs

### Probability of detection (hit rate): What fraction of the observed "yes" events were correctly forecast?

In [7]:
print(f'{((tp)/(tp+fn))*100:.2f}% of all observed LLJs were forecasted')

29.66% of all observed LLJs were forecasted


### False alarm ratio: What fraction of the predicted "yes" events actually did not occur (i.e., were false alarms)?
- should be used with above

In [8]:
print(f'{((fp)/(tp+fp))*100:.2f}% of forecasts were false alarms')

60.55% of forecasts were false alarms


### Probability of false detection (false alarm rate): What fraction of the observed "no" events were incorrectly forecast as "yes"?

In [9]:
print(f'{((fp)/(tn+fp))*100:.2f}% of observed no LLJs were forecasted as yes')

0.79% of observed no LLJs were forecasted as yes


### Success ratio: What fraction of the forecast "yes" events were correctly observed?

In [10]:
print(f'{((tp)/(tp+fp))*100:.2f}% of forecasted LLJs were observed')

39.45% of forecasted LLJs were observed


### Threat score (critical success index): How well did the forecast "yes" events correspond to the observed "yes" events?

In [11]:
print(f'{((tp)/(tp+fn+fp))*100:.2f}% of LLJs were correctly forecasted')

20.38% of LLJs were correctly forecasted


### Equitable threat score (Gilbert kill score): How well did the forecast "yes" events correspond to the observed "yes" events (accounting for hits due to chance)?

In [12]:
hits_rand = ((tp+fn)*(tp+fp)) / ttl
print(f'{((tp-hits_rand)/(tp+fn+fp-hits_rand))}')

0.19672601485286104


some skill in forecasting LLJs

### Hanssen and Kuipers discriminant (true skill statistic, Peirce's skill score): How well did the forecast separate the "yes" events from the "no" events?
- the Hanssen and Kuipers score can also be interpreted as (accuracy for events) + (accuracy for non-events) - 1. For rare events HK is unduly weighted toward the first term (same as POD), so this score may be more useful for more frequent events.

In [13]:
p1 = tp / (tp+fn)
p2 = fp / (fp+tn)
print(f'{p1-p2}')

0.2886673614572477
